# Support Vector Machines

Метод опорных векторов (SVM) — это особенно мощный и гибкий класс контролируемых алгоритмов как для классификации, так и для регрессии.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

## Идея

In [ ]:
from sklearn.datasets import make_blobs
X, y = make_blobs(n_samples=50, centers=2,
                  random_state=0, cluster_std=0.60)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn');

Линейный классификатор попытается провести прямую линию, разделяющую два набора данных, и тем самым создать модель для классификации.
Для двумерных данных, подобных показанным здесь, это задача, которую мы могли бы выполнить вручную.
Но мы сразу же видим проблему: существует более одной возможной разделительной линии, которая может идеально различать два класса.

In [ ]:
xfit = np.linspace(-1, 3.5)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
plt.plot([0.6], [2.1], 'x', color='red', markeredgewidth=2, markersize=10)

for m, b in [(1, 0.65), (0.5, 1.6), (-0.2, 2.9)]:
    plt.plot(xfit, m * xfit + b, '-k')

plt.xlim(-1, 3.5);

Это три разных разделителя, которые, тем не менее, хорошо различают эти примеры.
В зависимости от того, какой из них вы выберете, новой точке данных (например, отмеченной «X» на этом графике) будет присвоена разная метка.
Очевидно, что наша простая интуиция «проведения линии между классами» недостаточно хороша.

## Метод опорных векторов: максимизация отступа

Метод опорных векторов предлагают один из способов улучшить это.
Интуиция такова: вместо того, чтобы просто рисовать линию нулевой ширины между классами, мы можем нарисовать вокруг каждой линии *отступ* некоторой ширины, до ближайшей точки.

In [ ]:
xfit = np.linspace(-1, 3.5)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')

for m, b, d in [(1, 0.65, 0.33), (0.5, 1.6, 0.55), (-0.2, 2.9, 0.2)]:
    yfit = m * xfit + b
    plt.plot(xfit, yfit, '-k')
    plt.fill_between(xfit, yfit - d, yfit + d, edgecolor='none',
                     color='lightgray', alpha=0.5)

plt.xlim(-1, 3.5);

Линию, которая максимизирует этот отступ, мы и выберем в качестве оптимальной модели.

### Обучени метода опорных векторов

Будем использовать классификатор опорных векторов Scikit-Learn (`SVC`) в качестве модели SVM на этих данных.
На данный момент мы будем использовать линейное ядро ​​и установим параметр `C` на очень большое число.

In [ ]:
from sklearn.svm import SVC # "Support vector classifier"
model = SVC(kernel='linear', C=1E10)
model.fit(X, y)

Функция, которая построит для нас границы отступа SVM:

In [9]:
def plot_svc_decision_function(model, ax=None, plot_support=True):
    """Plot the decision function for a 2D SVC"""
    if ax is None:
        ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    # create grid to evaluate model
    x = np.linspace(xlim[0], xlim[1], 30)
    y = np.linspace(ylim[0], ylim[1], 30)
    Y, X = np.meshgrid(y, x)
    xy = np.vstack([X.ravel(), Y.ravel()]).T
    P = model.decision_function(xy).reshape(X.shape)

    # plot decision boundary and margins
    ax.contour(X, Y, P, colors='k',
               levels=[-1, 0, 1], alpha=0.5,
               linestyles=['--', '-', '--'])

    # plot support vectors
    if plot_support:
        ax.scatter(model.support_vectors_[:, 0],
                   model.support_vectors_[:, 1],
                   s=300, linewidth=1, edgecolors='black',
                   facecolors='none');
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
plot_svc_decision_function(model);

Это разделительная линия, которая максимизирует разницу между двумя наборами точек.
Обратите внимание, что несколько точек обучения просто касаются границы.

Эти точки являются основными элементами для обучения. Это *опорные векторы* дающие алгоритму его название.
В Scikit-Learn идентификаторы этих точек хранятся в атрибуте `support_vectors_` классификатора.

In [ ]:
model.support_vectors_

Ключом к успеху этого классификатора является то, что для обучения важны только положения опорных векторов, а любые точки, находящиеся дальше от границы, но на правильной стороне, не влияют на обучение.

Технически это происходит потому, что эти точки не вносят вклад в функцию потерь, используемую для подгонки модели, поэтому их положение и количество не имеют значения, пока они не пересекают границу.

Мы можем увидеть это, например, если построим график модели, обученной на основе первых 60 точек и первых 120 точек этого набора данных.

In [ ]:
def plot_svm(N=10, ax=None):
    X, y = make_blobs(n_samples=200, centers=2,
                      random_state=0, cluster_std=0.60)
    X = X[:N]
    y = y[:N]
    model = SVC(kernel='linear', C=1E10)
    model.fit(X, y)

    ax = ax or plt.gca()
    ax.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
    ax.set_xlim(-1, 4)
    ax.set_ylim(-1, 6)
    plot_svc_decision_function(model, ax)

fig, ax = plt.subplots(1, 2, figsize=(16, 6))
fig.subplots_adjust(left=0.0625, right=0.95, wspace=0.1)
for axi, N in zip(ax, [60, 120]):
    plot_svm(N, axi)
    axi.set_title('N = {0}'.format(N))

Эта нечувствительность к точному поведению удаленных точек является одной из сильных сторон модели SVM.

Виджеты IPython для интерактивного просмотра этой функции модели SVM.

In [ ]:
from ipywidgets import interact, fixed
interact(plot_svm, N=(10, 200), ax=fixed(None));

### Ядра SVM 

Возможности SVM могут быть расширены, за счет использования *ядер*.

In [ ]:
from sklearn.datasets import make_circles
X, y = make_circles(100, factor=.1, noise=.1)

clf = SVC(kernel='linear').fit(X, y)

plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
plot_svc_decision_function(clf, plot_support=False);

Очевидно, что никакая линейная дискриминация *никогда* не сможет разделить эти данные.

Здесь может бють использована *радиальная базисная функция* (RBF) с центром в средней группе.

In [15]:
r = np.exp(-(X ** 2).sum(1))

Мы можем визуализировать это дополнительное измерение данных с помощью трехмерного графика.

In [ ]:
from mpl_toolkits import mplot3d

ax = plt.subplot(projection='3d')
ax.scatter3D(X[:, 0], X[:, 1], r, c=y, s=50, cmap='autumn')
ax.view_init(elev=20, azim=30)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('r');

Мы видим, что с этим дополнительным измерением данные становятся тривиально линейно разделимыми, если нарисовать разделяющую плоскость, скажем, при *r*=0,7.

В этом случае нам пришлось выбрать и тщательно настроить нашу проекцию: если бы мы не центрировали нашу радиальную базисную функцию в правильном месте, мы бы не увидели таких чистых, линейно разделимых результатов.
В общем, необходимость делать такой выбор является проблемой: мы хотели бы каким-то образом автоматически находить лучшие базисные функции для использования.

Одна из стратегий для этого — вычислить базисную функцию, центрированную в *каждой* точке в наборе данных, и позволить алгоритму SVM работать с этими результатами.

Этот тип преобразования базисной функции известен как *ядерное преобразование*, поскольку он основан на отношении подобия (или ядре) между каждой парой точек.

Потенциальная проблема с этой стратегией — проецированием $N$ точек в $N$ измерений — заключается в том, что она может стать очень вычислительно дорогой по мере увеличения $N$.
Однако благодаря небольшой аккуратной процедуре, известной как [*ядерный трюк*](https://en.wikipedia.org/wiki/Kernel_trick), подгонка данных, преобразованных ядром, может быть выполнена неявно, то есть без построения полного $N$-мерного представления проекции ядра.

Этот трюк встроен в SVM и является одной из причин, по которой этот метод настолько эффективен.

В Scikit-Learn мы можем применить SVM с ядром, просто изменив наше линейное ядро ​​на ядро ​​RBF, используя гиперпараметр модели `kernel`.

In [ ]:
clf = SVC(kernel='rbf', C=1E6)
clf.fit(X, y)

Давайте используем нашу ранее определенную функцию для визуализации подгонки и определения опорных векторов.

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
plot_svc_decision_function(clf)
plt.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1],
            s=300, lw=1, facecolors='none');

Эта стратегия преобразования ядра часто используется в машинном обучении для превращения быстрых линейных методов в быстрые нелинейные методы, особенно для моделей, в которых можно использовать трюк с ядром.

### Настройка SVM: смягчение отступа

До сих пор наше обсуждение было сосредоточено вокруг очень чистых наборов данных, в которых существует идеальная граница разделения.

Но что, если ваши данные имеют некоторую степень перекрытия?

In [ ]:
X, y = make_blobs(n_samples=100, centers=2,
                  random_state=0, cluster_std=1.2)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn');

Чтобы справиться с этим случаем, реализация SVM имеет немного фактора подгонки, который «смягчает» границу: то есть, она позволяет некоторым точкам проникать в границу, если это позволяет лучше подогнать ее.
Жесткость границы контролируется параметром настройки `C`.
Для очень большого `C` граница жесткая, и точки не могут лежать в ней.
Для меньшего `C` граница мягче и может расширяться, охватывая некоторые точки.

In [ ]:
X, y = make_blobs(n_samples=100, centers=2,
                  random_state=0, cluster_std=0.8)

fig, ax = plt.subplots(1, 2, figsize=(16, 6))
fig.subplots_adjust(left=0.0625, right=0.95, wspace=0.1)

for axi, C in zip(ax, [10.0, 0.1]):
    model = SVC(kernel='linear', C=C).fit(X, y)
    axi.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
    plot_svc_decision_function(model, axi)
    axi.scatter(model.support_vectors_[:, 0],
                model.support_vectors_[:, 1],
                s=300, lw=1, facecolors='none');
    axi.set_title('C = {0:.1f}'.format(C), size=14)

## Пример: Распознавание лиц

В качестве примера для метода опорных векторов рассмотрим задачу распознавания лиц.

Мы будем использовать набор данных Labeled Faces in the Wild, который состоит из нескольких тысяч сопоставленных фотографий различных публичных личностей.

In [ ]:
from sklearn.datasets import fetch_lfw_people
faces = fetch_lfw_people(min_faces_per_person=60)
print(faces.target_names)
print(faces.images.shape)

In [ ]:
fig, ax = plt.subplots(3, 5, figsize=(8, 6))
for i, axi in enumerate(ax.flat):
    axi.imshow(faces.images[i], cmap='bone')
    axi.set(xticks=[], yticks=[],
            xlabel=faces.target_names[faces.target[i]])

Мы могли бы продолжить, просто используя каждое значение пикселя как признак, но часто бывает эффективнее использовать какой-либо препроцессор для извлечения более значимых признаков.

In [23]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline

pca = PCA(n_components=150, whiten=True,
          svd_solver='randomized', random_state=42)
svc = SVC(kernel='rbf', class_weight='balanced')
model = make_pipeline(pca, svc)

In [24]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(faces.data, faces.target,
                                                random_state=42)

Мы можем использовать перекрестную проверку (кросс валидацию) поиска по сетке для изучения комбинаций параметров.

Здесь мы настроим `C` (который управляет жесткостью границ) и `gamma` (который управляет размером ядра радиальной базисной функции) и определим лучшую модель.

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'svc__C': [1, 5, 10, 50],
              'svc__gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid)

%time grid.fit(Xtrain, ytrain)
print(grid.best_params_)

Оптимальные значения попадают в середину нашей сетки; если бы они попадали на края, нам бы хотелось расширить сетку, чтобы убедиться, что мы нашли истинный оптимум.

In [26]:
model = grid.best_estimator_
yfit = model.predict(Xtest)

Давайте рассмотрим несколько тестовых изображений вместе с их прогнозируемыми значениями.

In [ ]:
fig, ax = plt.subplots(4, 6)
for i, axi in enumerate(ax.flat):
    axi.imshow(Xtest[i].reshape(62, 47), cmap='bone')
    axi.set(xticks=[], yticks=[])
    axi.set_ylabel(faces.target_names[yfit[i]].split()[-1],
                   color='black' if yfit[i] == ytest[i] else 'red')
fig.suptitle('Predicted Names; Incorrect Labels in Red', size=14);

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(ytest, yfit,
                            target_names=faces.target_names))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
mat = confusion_matrix(ytest, yfit)
sns.heatmap(mat.T, square=True, annot=True, fmt='d',
            cbar=False, cmap='Blues',
            xticklabels=faces.target_names,
            yticklabels=faces.target_names)
plt.xlabel('true label')
plt.ylabel('predicted label');

Для реальной задачи распознавания лиц, в которой фотографии не обрезаются заранее, единственное отличие в схеме классификации лиц заключается в выборе признаков: нужно будет использовать более сложный алгоритм для поиска лиц и извлечения признаков.
Для такого рода приложений одним из вариантов является использование [OpenCV](http://opencv.org), который включает предварительно обученные реализации современных инструментов извлечения признаков для изображений в целом и лиц в частности.